### Job Extraction Pipeline


In [1]:
import pandas as pd
import json
import time
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

# Import helper functions
from jobs_helper import scrape_single_config

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# File paths
SITE_CONFIGS_PATH = './data/site_configs_raw.json'
OUTPUT_JOBS_CSV = './data/avature_jobs.csv'
OUTPUT_JOBS_JSON = './data/avature_jobs.json'

# Scraping configuration
MAX_WORKERS = 5

print("Configuration loaded")
print(f"Input: {SITE_CONFIGS_PATH}")
print(f"Output: {OUTPUT_JOBS_CSV}")


Configuration loaded
Input: ./data/site_configs_raw.json
Output: ./data/avature_jobs.csv


In [2]:
# Load site configurations
with open(SITE_CONFIGS_PATH, 'r') as f:
    all_configs = json.load(f)

df_configs = pd.DataFrame(all_configs)

print("="*60)
print("SITE CONFIGURATIONS LOADED")
print("="*60)

# Filter for scrapable endpoints
success_configs = df_configs[df_configs['status'] == 'success'].to_dict('records')
partial_configs = df_configs[df_configs['status'] == 'partial'].to_dict('records')

print(f"\nEndpoint Status:")
print(f"  SUCCESS (paginated): {len(success_configs)}")
print(f"  PARTIAL (HTML scrape): {len(partial_configs)}")
print(f"  Total to scrape: {len(success_configs) + len(partial_configs)}")

print(f"\nSample SUCCESS endpoints:")
for config in success_configs[:3]:
    print(f"  {config['tenant']}: {config['endpoint']}")

if len(partial_configs) > 0:
    print(f"\nSample PARTIAL endpoints:")
    for config in partial_configs[:3]:
        print(f"  {config['tenant']}: {config.get('sample_job_ids', 'N/A')}")


SITE CONFIGURATIONS LOADED

Endpoint Status:
  SUCCESS (paginated): 179
  PARTIAL (HTML scrape): 9
  Total to scrape: 188

Sample SUCCESS endpoints:
  ally.avature.net: https://ally.avature.net/careers/SearchJobs
  a2milkkf.avature.net: https://a2milkkf.avature.net/careers/SearchJobs
  amerilife.avature.net: https://amerilife.avature.net/careers/SearchJobs

Sample PARTIAL endpoints:
  arcbest.avature.net: ['27800', '27986', '27573', '27984', '27985']
  baufest.avature.net: ['5773', '5763', '5779', '5775', '5767']
  deltaflightattendants.avature.net: ['15', '16']


In [3]:
print("="*60)
print("STARTING JOB EXTRACTION")
print("="*60)

all_jobs = []
configs_to_scrape = success_configs + partial_configs

print(f"\nProcessing {len(configs_to_scrape)} endpoints...")
print(f"Estimated time: {len(configs_to_scrape) * 30 / 60:.1f} minutes\n")

start_time = time.time()

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_config = {
        executor.submit(scrape_single_config, config): config 
        for config in configs_to_scrape
    }
    
    completed = 0
    for future in as_completed(future_to_config):
        config = future_to_config[future]
        try:
            jobs = future.result()
            all_jobs.extend(jobs)
            completed += 1
            
            if completed % 10 == 0:
                elapsed = time.time() - start_time
                jobs_so_far = len(all_jobs)
                print(f"\nProgress: {completed}/{len(configs_to_scrape)} sites | "
                      f"{jobs_so_far:,} jobs | {elapsed/60:.1f} min elapsed")
        except Exception as e:
            logger.error(f"Future failed for {config['tenant']}: {e}")

elapsed_time = time.time() - start_time

print("\n" + "="*60)
print("SCRAPING COMPLETE")
print("="*60)
print(f"\nTotal time: {elapsed_time/60:.1f} minutes")
print(f"Total jobs collected: {len(all_jobs):,}")
print(f"Sites scraped: {len(configs_to_scrape)}")
if len(configs_to_scrape) > 0:
    print(f"Average: {len(all_jobs)/len(configs_to_scrape):.1f} jobs/site")


2026-02-08 15:50:48,222 - INFO - Scraping ally.avature.net...
2026-02-08 15:50:48,240 - INFO - Scraping a2milkkf.avature.net...
2026-02-08 15:50:48,245 - INFO - Scraping amerilife.avature.net...
2026-02-08 15:50:48,245 - INFO - Scraping amswh.avature.net...
2026-02-08 15:50:48,247 - INFO - Scraping advocateaurorahealth.avature.net...


STARTING JOB EXTRACTION

Processing 188 endpoints...
Estimated time: 94.0 minutes



2026-02-08 15:50:52,732 - INFO - ally.avature.net: Page 1 - 28 new jobs, 11 duplicates (total: 28)
2026-02-08 15:50:53,715 - INFO - advocateaurorahealth.avature.net: Page 1 - 30 new jobs, 11 duplicates (total: 30)
2026-02-08 15:50:53,891 - INFO - amerilife.avature.net: Page 1 - 23 new jobs, 10 duplicates (total: 23)
2026-02-08 15:50:55,515 - INFO - a2milkkf.avature.net: Page 1 - 33 new jobs, 0 duplicates (total: 33)
2026-02-08 15:50:55,723 - INFO - amswh.avature.net: Page 1 - 23 new jobs, 9 duplicates (total: 23)
2026-02-08 15:50:57,734 - INFO - advocateaurorahealth.avature.net: Page 2 - 35 new jobs, 12 duplicates (total: 65)
2026-02-08 15:50:57,965 - INFO - ally.avature.net: Page 2 - 27 new jobs, 12 duplicates (total: 55)
2026-02-08 15:50:59,297 - INFO - amerilife.avature.net: Page 2 - 23 new jobs, 10 duplicates (total: 46)
2026-02-08 15:51:00,495 - INFO - a2milkkf.avature.net: Page 2 - 3 new jobs, 0 duplicates (total: 36)
2026-02-08 15:51:01,219 - INFO - ally.avature.net: Page 3 - 3 


Progress: 10/188 sites | 1,765 jobs | 3.4 min elapsed


2026-02-08 15:54:10,812 - INFO - bmcrecruit.avature.net: Page 2 - 25 new jobs, 6 duplicates (total: 44)
2026-02-08 15:54:11,316 - INFO - bloomberg.avature.net: Page 4 - 13 new jobs, 13 duplicates (total: 55)
2026-02-08 15:54:11,787 - INFO - auspost.avature.net: Page 4 - 15 new jobs, 0 duplicates (total: 153)
2026-02-08 15:54:13,423 - INFO - berenberg.avature.net: Page 5 - 6 new jobs, 0 duplicates (total: 50)
2026-02-08 15:54:13,954 - INFO - bradyplus.avature.net: Page 1 - 13 new jobs, 6 duplicates (total: 13)
2026-02-08 15:54:13,994 - INFO - bmcrecruit.avature.net: Page 3 - 20 new jobs, 5 duplicates (total: 64)
2026-02-08 15:54:15,216 - INFO - bloomberg.avature.net: Page 5 - 13 new jobs, 13 duplicates (total: 68)
2026-02-08 15:54:16,443 - INFO - auspost.avature.net: Page 5 - 15 new jobs, 0 duplicates (total: 168)
2026-02-08 15:54:18,354 - INFO - bmcrecruit.avature.net: Page 4 - 14 new jobs, 0 duplicates (total: 78)
2026-02-08 15:54:18,434 - INFO - bradyplus.avature.net: Page 2 - 19 new

...


Progress: 170/188 sites | 44,802 jobs | 59.7 min elapsed


2026-02-08 16:50:30,049 - INFO - voutiqueintegrations.avature.net: Page 9 - 4 new jobs, 0 duplicates (total: 36)
2026-02-08 16:50:30,473 - INFO - voutiquetraining.avature.net: Page 9 - 3 new jobs, 0 duplicates (total: 27)
2026-02-08 16:50:32,306 - INFO - walmartsourcingeu.avature.net: Page 5 - 2 new jobs, 0 duplicates (total: 10)
2026-02-08 16:50:32,521 - INFO - voutiqueintegrations.avature.net: Page 16 - 4 new jobs, 0 duplicates (total: 64)
2026-02-08 16:50:35,382 - INFO - wickes.avature.net: Page 1 - 23 new jobs, 20 duplicates (total: 23)
2026-02-08 16:50:35,737 - INFO - voutiqueintegrations.avature.net: Page 10 - 4 new jobs, 0 duplicates (total: 40)
2026-02-08 16:50:35,737 - INFO - voutiquetraining.avature.net: Page 10 - 3 new jobs, 0 duplicates (total: 30)
2026-02-08 16:50:35,986 - INFO - walmartsourcingeu.avature.net: Page 6 - 2 new jobs, 0 duplicates (total: 12)
2026-02-08 16:50:37,341 - INFO - voutiqueintegrations.avature.net: Page 17 - 4 new jobs, 0 duplicates (total: 68)
2026-


Progress: 180/188 sites | 45,190 jobs | 62.0 min elapsed


2026-02-08 16:52:50,548 - INFO - zungfu.avature.net: Page 13 - 4 new jobs, 0 duplicates (total: 88)
2026-02-08 16:52:51,411 - INFO - workmyway.avature.net: Page 11 - 18 new jobs, 0 duplicates (total: 367)
2026-02-08 16:52:54,224 - INFO - xerox.avature.net: Page 14 - 6 new jobs, 0 duplicates (total: 264)
2026-02-08 16:52:55,969 - INFO - zungfu.avature.net: Page 14 - 4 new jobs, 0 duplicates (total: 92)
2026-02-08 16:52:59,654 - INFO - xerox.avature.net: Page 15 - 6 new jobs, 0 duplicates (total: 270)
2026-02-08 16:52:59,967 - INFO - workmyway.avature.net: Page 12 - 18 new jobs, 0 duplicates (total: 385)
2026-02-08 16:53:01,296 - INFO - zungfu.avature.net: Page 15 - 4 new jobs, 0 duplicates (total: 96)
2026-02-08 16:53:05,354 - INFO - xerox.avature.net: Page 16 - 6 new jobs, 0 duplicates (total: 276)
2026-02-08 16:53:06,171 - INFO - delta.avature.net: Collected 5 jobs from partial
2026-02-08 16:53:06,173 - INFO - Scraping jawood.avature.net (partial - 1 jobs)...
2026-02-08 16:53:06,614 -


SCRAPING COMPLETE

Total time: 63.2 minutes
Total jobs collected: 46,151
Sites scraped: 188
Average: 245.5 jobs/site


In [7]:
df_jobs = pd.DataFrame(all_jobs)

print("\n" + "="*60)
print("DEDUPLICATION")
print("="*60)

before_dedup = len(df_jobs)
print(f"\nOriginal jobs collected: {before_dedup:,}")

# Step 1: Remove same-tenant duplicates
df_jobs_step1 = df_jobs.drop_duplicates(subset=['tenant', 'job_id'], keep='first')
removed_step1 = before_dedup - len(df_jobs_step1)
print(f"Step 1 - Same-tenant duplicates: {removed_step1:,} removed")

# Step 2: Remove cross-tenant duplicates
df_jobs_step2 = df_jobs_step1.drop_duplicates(subset=['job_id'], keep='first')
removed_step2 = len(df_jobs_step1) - len(df_jobs_step2)
print(f"Step 2 - Cross-tenant duplicates: {removed_step2:,} removed")

# Step 3: Remove URL duplicates
df_jobs_final = df_jobs_step2.drop_duplicates(subset=['job_url'], keep='first')
removed_step3 = len(df_jobs_step2) - len(df_jobs_final)
print(f"Step 3 - URL duplicates: {removed_step3:,} removed")

print(f"\nFinal result:")
print(f"  Before: {before_dedup:,}")
print(f"  After: {len(df_jobs_final):,}")
print(f"  Removed: {before_dedup - len(df_jobs_final):,} ({(before_dedup - len(df_jobs_final))/before_dedup*100:.1f}%)")

df_jobs = df_jobs_final

# Save files
df_jobs.to_csv(OUTPUT_JOBS_CSV, index=False)
print(f"\nSaved CSV: {OUTPUT_JOBS_CSV}")

with open(OUTPUT_JOBS_JSON, 'w') as f:
    json.dump(df_jobs.to_dict('records'), f, indent=2)
print(f"Saved JSON: {OUTPUT_JOBS_JSON}")



DEDUPLICATION

Original jobs collected: 46,151
Step 1 - Same-tenant duplicates: 40,196 removed
Step 2 - Cross-tenant duplicates: 325 removed
Step 3 - URL duplicates: 0 removed

Final result:
  Before: 46,151
  After: 5,630
  Removed: 40,521 (87.8%)

Saved CSV: ./data/avature_jobs.csv
Saved JSON: ./data/avature_jobs.json


In [8]:
print("\n" + "="*60)
print("DATA QUALITY REPORT")
print("="*60)

# Check completeness
print(f"\nField Completeness:")
for col in ['job_title', 'job_url', 'job_id', 'location', 'job_description']:
    completeness = df_jobs[col].notna().sum() / len(df_jobs) * 100
    print(f"  {col}: {completeness:.1f}%")

# Check for issues
issues = []

no_title = df_jobs[df_jobs['job_title'].isna()]
if len(no_title) > 0:
    issues.append(f"{len(no_title)} jobs missing titles")

no_url = df_jobs[df_jobs['job_url'].isna()]
if len(no_url) > 0:
    issues.append(f"{len(no_url)} jobs missing URLs")

if issues:
    print(f"\nIssues Found:")
    for issue in issues:
        print(f"  {issue}")
else:
    print(f"\nNo critical issues found")



DATA QUALITY REPORT

Field Completeness:
  job_title: 100.0%
  job_url: 100.0%
  job_id: 100.0%
  location: 27.6%
  job_description: 0.4%

No critical issues found
